In [1]:
import pandas as pd
import requests
import os
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [2]:
for dir_name in ['data', 'data/gameweek_scores', 'data/match_stats', 'data/player_stats']:
    os.makedirs(dir_name, exist_ok=True)

base_url = "https://api.sofascore.com/api/v1/unique-tournament/17/season/41886/"
current_gw = 8

### Team

##### Overall

In [3]:
r = requests.get(base_url + "standings/total").json()
stats = r['standings'][0]['rows']

team_stats_overall = pd.DataFrame([{
    'name': team['team']['name'],
    'position': team['position'],
    'matches': team['matches'],
    'wins': team['wins'],
    'losses': team['losses'],
    'draws': team['draws'],
    'scoresFor': team['scoresFor'],
    'scoresAgainst': team['scoresAgainst'],
    'points': team['points']
} for team in stats])

##### Home

In [4]:
r = requests.get(base_url + "standings/home").json()
stats = r['standings'][0]['rows']

team_stats_home = pd.DataFrame([{
    'name': team['team']['name'],
    'position': team['position'],
    'matches': team['matches'],
    'wins': team['wins'],
    'losses': team['losses'],
    'draws': team['draws'],
    'scoresFor': team['scoresFor'],
    'scoresAgainst': team['scoresAgainst'],
    'points': team['points']
} for team in stats])

##### Away

In [5]:
r = requests.get(base_url + "standings/away").json()
stats = r['standings'][0]['rows']

team_stats_away = pd.DataFrame([{
    'name': team['team']['name'],
    'position': team['position'],
    'matches': team['matches'],
    'wins': team['wins'],
    'losses': team['losses'],
    'draws': team['draws'],
    'scoresFor': team['scoresFor'],
    'scoresAgainst': team['scoresAgainst'],
    'points': team['points']

} for team in stats])

In [6]:
team_stats_overall

,name,position,matches,wins,losses,draws,scoresFor,scoresAgainst,points
0,Arsenal,1,7,6,1,0,17,7,18
1,Manchester City,2,7,5,0,2,23,6,17
2,Tottenham Hotspur,3,7,5,0,2,18,7,17
3,Brighton & Hove Albion,4,6,4,1,1,11,5,13
4,Manchester United,5,6,4,2,0,8,8,12
5,Fulham,6,7,3,2,2,12,11,11
6,Chelsea,7,6,3,2,1,8,9,10
7,Liverpool,8,6,2,1,3,15,6,9
8,Brentford,9,7,2,2,3,15,12,9
9,Newcastle United,10,7,1,1,5,8,7,8


### Gameweek

In [7]:
def get_gameweek_matches(gw_no):
    try:
        gw_score = pd.read_json('data/gameweek_scores/scores_gw{}.json'.format(gw_no))
    except ValueError:
        r = requests.get(base_url + "events/round/{}".format(gw_no)).json()
        gw_score = pd.DataFrame([{
            'gameWeek': gw_no,
            'matchId': event['id'],
            'homeTeam': event['homeTeam']['name'],
            'homeScore': event['homeScore']['current'],
            'awayScore': event['awayScore']['current'],
            'awayTeam': event['awayTeam']['name'],
            'scoreCombined': '{}-{}'.format(event['homeScore']['current'], event['awayScore']['current'])
        } for event in r['events'] if event['status']['code'] == 100])
        with open('data/gameweek_scores/scores_gw{}.json'.format(gw_no), 'w') as f:
            gw_score.to_json(f, orient='records')
    return gw_score

gw_scores_all = pd.concat([get_gameweek_matches(gw_no) for gw_no in range(1, current_gw+1)])

### Match

In [8]:
def get_match_stats(row):
    match_id = row['matchId']
    try:
        df = pd.read_json('data/match_stats/stats_match{}.json'.format(match_id))
    except ValueError:
        r = requests.get('https://api.sofascore.com/api/v1/event/{}/statistics'.format(match_id)).json()
        stats = []
        for x in r['statistics'][0]['groups']:
            if x['groupName'] not in ['TVData', 'Passes', 'Duels']:
                stats.extend([{
                    'category': x['groupName'],
                    'name': y['name'],
                    'home': y['home'],
                    'away': y['away']
                } for y in x['statisticsItems']])
        df = pd.DataFrame(stats)
        for k, v in row.items():
            df[k] = v
        with open('data/match_stats/stats_match{}.json'.format(match_id), 'w') as f:
            df.to_json(f, orient='records')
    return df[['gameWeek', 'matchId', 'homeTeam', 'homeScore', 'awayScore', 'awayTeam', 'category', 'name', 'home', 'away']]

team_stats_scores_gw_all = pd.concat([get_match_stats(r) for i, r in gw_scores_all.iterrows()])

### Player

In [9]:
def get_player_stats_in_match(row):
    try:
        df = pd.read_json('data/player_stats/stats_players_match{}.json'.format(row['matchId']))
    except:
        r = requests.get('https://api.sofascore.com/api/v1/event/{}/lineups'.format(row['matchId'])).json()
        player_details = []
        for loc in ['home', 'away']:
            for player in r[loc]['players']:
                d = row.to_dict()
                d.update({
                    'playerId': player['player']['id'],
                    'playerName': player['player']['name'],
                    'teamName': row['{}Team'.format(loc)],
                    'playerPosition': player['player']['position'],
                    'playingOn': loc.title(),
                    'subsititute': player['substitute']
                })
                d.update(player['statistics'])
                player_details.append(d)
        df = pd.DataFrame(player_details)
        with open('data/player_stats/stats_players_match{}.json'.format(row['matchId']), 'w') as f:
            df.to_json(f, orient='records')
    return df

player_stats_gw_all = pd.concat([get_player_stats_in_match(row) for i, row in gw_scores_all.iterrows()])

### Player Statistics

In [10]:
COLUMNS_G = [
    'gameWeek', 'homeTeam', 'homeScore', 'awayScore', 'awayTeam', 'playerName', 'teamName', 'playerPosition',
    'playingOn', 'subsititute', 'minutesPlayed', 'rating', 'actualFPLPoints', 'possibleFPLPoints',
    'savedShotsFromInsideTheBox', 'saves', 'totalKeeperSweeper', 'accurateKeeperSweeper', 'possessionLostCtrl',
    'keyPass', 'aerialWon', 'aerialLost', 'duelWon', 'duelLost', 'interceptionWon', 'wasFouled', 'fouls',
    'bigChanceCreated', 'goodHighClaim', 'punches', 'goalAssist', 'ownGoals', 'errorLeadToAGoal',
    'errorLeadToAShot', 'penaltySave', 'penaltyConceded'
]
COLUMNS_NG = [
    'gameWeek', 'homeTeam', 'homeScore', 'awayScore', 'awayTeam', 'playerName', 'teamName', 'playerPosition',
    'playingOn', 'subsititute', 'minutesPlayed', 'rating', 'actualFPLPoints', 'possibleFPLPoints', 'goals',
    'goalAssist', 'goalThreat', 'assistThreat', 'hitWoodwork', 'shotOffTarget', 'bigChanceMissed',
    'onTargetScoringAttempt', 'penaltyWon', 'penaltyMiss', 'bigChanceCreated', 'keyPass', 'touches',
    'accurateCross', 'totalCross', 'totalOffside', 'wasFouled', 'possessionLostCtrl', 'aerialWon',
    'aerialLost', 'duelWon', 'duelLost', 'dispossessed', 'totalContest', 'wonContest', 'clearanceOffLine',
    'interceptionWon', 'totalTackle', 'lastManTackle', 'fouls', 'blockedScoringAttempt', 'challengeLost',
    'ownGoals', 'errorLeadToAGoal', 'errorLeadToAShot', 'penaltyConceded'
]

In [11]:
def calculate_actual_attack_points(row):
    pts = row['goalAssist'] * 3
    if row['playerPosition'] == 'F':
        pts += (row['goals'] * 4)
    elif row['playerPosition'] == 'M':
        pts += (row['goals'] * 5)
    elif row['playerPosition'] == 'D':
        pts += (row['goals'] * 6)
    return pts

def calculate_possible_attack_points(row):
    pts = row['assistThreat'] * 3
    if row['playerPosition'] == 'F':
        pts += (row['goalThreat'] * 4)
    elif row['playerPosition'] == 'M':
        pts += (row['goalThreat'] * 5)
    elif row['playerPosition'] == 'D':
        pts += (row['goalThreat'] * 6)
    return pts

In [12]:
def get_player_stats(stats, playerPosition='NG', minutesPlayed=1, gameweeks=[], matchId=None):
    df = stats.fillna(0)
    df['goalThreat'] = df['goals'] + df['hitWoodwork'] + df['bigChanceMissed']
    df['assistThreat'] = df['goalAssist'] + df['penaltyWon'] + df['bigChanceCreated']
    df['actualFPLPoints'] = df.apply(lambda x: calculate_actual_attack_points(x), axis=1)
    df['possibleFPLPoints'] = df.apply(lambda x: calculate_possible_attack_points(x), axis=1)
    df = df[df.minutesPlayed >= minutesPlayed]
    df = df[df.playerPosition == playerPosition] if playerPosition != 'NG' else df[df.playerPosition != 'G']
    if gameweeks:
        df = df[df['gameWeek'].isin(gameweeks)]
    if matchId:
        df = df[df.matchId == matchId]
    return df[COLUMNS_G] if playerPosition == 'G' else df[COLUMNS_NG]

In [13]:
players_g = get_player_stats(player_stats_gw_all, 'G')  # Goal Keepers
players_ng = get_player_stats(player_stats_gw_all)  # All NG Players

In [14]:
def stats_filter(data, playerNames=[], playerPosition=None, teamName=None, location='All', gameWeeks=[]):
    filtered = data.copy()
    if playerNames:
        filtered = filtered[filtered['playerName'].isin(playerNames)]
    if playerPosition:
        filtered = filtered[filtered['playerPosition'] == playerPosition]
    if teamName:
        filtered = filtered[filtered['teamName'] == teamName]
    if location != 'All':
        filtered = filtered[filtered['playingOn'] == location]
    if gameWeeks:
        filtered = filtered[filtered['gameWeek'].isin(gameWeeks)]
    return filtered.drop('gameWeek', axis=1)

### Filter selected options on Home and/or Away

In [15]:
### Find player's full name
players_ng[players_ng.playerName.str.contains('bruyne', case=False)][['playerName', 'teamName']].drop_duplicates()

,playerName,teamName
25,Kevin De Bruyne,Manchester City


In [16]:
selected_players = ['Heung-min Son', 'Kevin De Bruyne']
selected_position = None
selected_team = None
selected_gameweeks = []

In [17]:
### Home and Away
filtered = stats_filter(players_ng, playerNames=selected_players, playerPosition=selected_position, teamName=selected_team, gameWeeks=selected_gameweeks)
filtered.groupby(['playerName']).agg(sum).sort_values(['possibleFPLPoints', 'actualFPLPoints', 'rating', 'minutesPlayed'], ascending=False).iloc[:50]

,homeScore,awayScore,subsititute,minutesPlayed,rating,actualFPLPoints,possibleFPLPoints,goals,goalAssist,goalThreat,assistThreat,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
playerName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Kevin De Bruyne,18,11,1,540.0,54.9,23.0,67.0,1.0,6.0,5.0,14.0,1.0,7.0,3.0,5.0,0.0,0.0,8.0,25.0,450.0,16.0,52.0,0.0,5.0,115.0,3.0,4.0,20.0,23.0,6.0,9.0,6.0,0.0,2.0,6.0,0.0,6.0,7.0,4.0,0.0,0.0,0.0,0.0
Heung-min Son,16,9,1,524.0,50.8,15.0,31.0,3.0,1.0,7.0,1.0,2.0,5.0,2.0,11.0,0.0,0.0,0.0,14.0,257.0,6.0,29.0,3.0,8.0,88.0,3.0,4.0,18.0,30.0,9.0,11.0,1.0,0.0,2.0,6.0,0.0,5.0,5.0,3.0,0.0,0.0,0.0,0.0


In [18]:
### Home
filtered = stats_filter(players_ng, playerNames=selected_players, playerPosition=selected_position, teamName=selected_team, location='Home', gameWeeks=selected_gameweeks)
filtered.groupby(['playerName']).agg(sum).sort_values(['possibleFPLPoints', 'actualFPLPoints', 'minutesPlayed'], ascending=False).iloc[:50]

,homeScore,awayScore,subsititute,minutesPlayed,rating,actualFPLPoints,possibleFPLPoints,goals,goalAssist,goalThreat,assistThreat,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
playerName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Heung-min Son,13,4,1,281.0,31.4,15.0,27.0,3.0,1.0,6.0,1.0,2.0,4.0,1.0,9.0,0.0,0.0,0.0,10.0,167.0,5.0,21.0,2.0,5.0,51.0,1.0,1.0,12.0,15.0,3.0,7.0,1.0,0.0,1.0,5.0,0.0,3.0,2.0,2.0,0.0,0.0,0.0,0.0
Kevin De Bruyne,14,2,1,200.0,23.3,8.0,16.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0,3.0,0.0,0.0,1.0,8.0,191.0,8.0,26.0,0.0,2.0,47.0,0.0,3.0,8.0,8.0,2.0,5.0,4.0,0.0,0.0,2.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0


In [19]:
### Away
filtered = stats_filter(players_ng, playerNames=selected_players, playerPosition=selected_position, teamName=selected_team, location='Away', gameWeeks=selected_gameweeks)
filtered.groupby(['playerName']).agg(sum).sort_values(['possibleFPLPoints', 'actualFPLPoints', 'minutesPlayed'], ascending=False).iloc[:50]

,homeScore,awayScore,subsititute,minutesPlayed,rating,actualFPLPoints,possibleFPLPoints,goals,goalAssist,goalThreat,assistThreat,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
playerName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Kevin De Bruyne,4,9,0,340.0,31.6,15.0,51.0,0.0,5.0,3.0,12.0,1.0,6.0,2.0,2.0,0.0,0.0,7.0,17.0,259.0,8.0,26.0,0.0,3.0,68.0,3.0,1.0,12.0,15.0,4.0,4.0,2.0,0.0,2.0,4.0,0.0,4.0,3.0,4.0,0.0,0.0,0.0,0.0
Heung-min Son,3,5,0,243.0,19.4,0.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,4.0,90.0,1.0,8.0,1.0,3.0,37.0,2.0,3.0,6.0,15.0,6.0,4.0,0.0,0.0,1.0,1.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0


### FPL Fixtures

In [20]:
fpl_static = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').json()
fpl_teams = pd.DataFrame(fpl_static['teams'])
fpl_teams.index = fpl_teams['id']
fpl_teams = fpl_teams['name'].to_dict()

fpl_fixtures = requests.get('https://fantasy.premierleague.com/api/fixtures/?future=1').json()
fpl_fixture_ratings = pd.DataFrame(fpl_fixtures)[['event', 'team_h', 'team_h_difficulty', 'team_a', 'team_a_difficulty']]

fpl_fixture_ratings['team_h'] = fpl_fixture_ratings['team_h'].replace(fpl_teams)
fpl_fixture_ratings['team_a'] = fpl_fixture_ratings['team_a'].replace(fpl_teams)

fixtures = []
for team in fpl_teams.values():
    d = {'teamName': team}
    result = fpl_fixture_ratings[(fpl_fixture_ratings['team_h'] == team) | (fpl_fixture_ratings['team_a'] == team)]
    result = result[result['event'].isin(range(current_gw+1, current_gw+6))]
    for _, fix in result.iterrows():
        if fix['team_h'] == team:
            d['GW{}'.format(fix['event'])] = '{}. {} (H)'.format(fix['team_h_difficulty'], fix['team_a'])
        else:
            d['GW{}'.format(fix['event'])] = '{}. {} (A)'.format(fix['team_a_difficulty'], fix['team_h'])
    fixtures.append(d)
    
fixture_df = pd.DataFrame(fixtures).fillna('-')

In [21]:
def difficulty_color(val):
    if val[0] == '1':
        return 'background-color:darkgreen;'
    elif val[0] == '2':
        return 'background-color:lightgreen;'
    elif val[0] == '3':
        return 'background-color:lightgrey;'
    elif val[0] == '4':
        return 'background-color:red; color:white;'
    elif val[0] == '5':
        return 'background-color:darkred; color:white;'
    else:
        return None

In [22]:
fixture_df.style.applymap(lambda v: difficulty_color(v))

,teamName,GW9,GW10,GW11,GW13,GW12
0,Arsenal,3. Spurs (H),4. Liverpool (H),2. Leeds (A),2. Southampton (A),-
1,Aston Villa,2. Leeds (A),2. Nott'm Forest (A),3. Chelsea (H),2. Brentford (H),2. Fulham (A)
2,Bournemouth,2. Brentford (H),2. Leicester (H),2. Fulham (A),3. West Ham (A),2. Southampton (H)
3,Brentford,2. Bournemouth (A),3. Newcastle (A),3. Brighton (H),2. Aston Villa (A),3. Chelsea (H)
4,Brighton,5. Liverpool (A),3. Spurs (H),3. Brentford (A),5. Man City (A),2. Nott'm Forest (H)
5,Chelsea,3. Crystal Palace (A),2. Wolves (H),2. Aston Villa (A),3. Man Utd (H),3. Brentford (A)
6,Crystal Palace,3. Chelsea (H),2. Leeds (H),2. Leicester (A),2. Everton (A),2. Wolves (H)
7,Everton,2. Southampton (A),3. Man Utd (H),4. Spurs (A),2. Crystal Palace (H),3. Newcastle (A)
8,Fulham,3. Newcastle (H),3. West Ham (A),2. Bournemouth (H),2. Leeds (A),2. Aston Villa (H)
9,Leicester,2. Nott'm Forest (H),2. Bournemouth (A),2. Crystal Palace (H),2. Wolves (A),2. Leeds (H)
